In [1]:
import json
import os
import pandas as pd
import numpy as np
from pathlib import Path
import collections
from sklearn.model_selection import train_test_split
from sklearn import metrics

import sys
sys.path.append("../")
from datatools.analyzer import *
from utterance.error_tools import *

from datatools.maneger import DataManager
from datatools.preproc import Preprocessor

In [25]:
path = "../eval_labeled/"
datalist = ['DCM', 'DIT', 'IRS']
convs = read_conv(path, datalist)

In [26]:
# 文章ごとに n-gram を考えてみる
def get_ngram_set(doc, N=3):
    if isinstance(doc, str):
        doc = nlp(doc)
    surfaces = [token.text for token in doc]
    ngram_set = set()
    filled = ["FOS", *surfaces, "EOS"]
    # print(filled)
    for i in range(len(filled)-N+1):
        f = "_".join(filled[i:i+N])
        ngram_set.add(f)
    return ngram_set

In [27]:
def check_repeat_rate(target:set, history:list, border=0.7):
    t_list = list(target)
    for prev_set in history:
        size = len(prev_set)
        hit = 0
        for t in t_list:
            if t in prev_set:
                hit+=1
        if hit/size >= border:
            return True
    
    return False

In [28]:
error = "Repetition"
y = []
for conv in convs:
    for ut in conv:
        if not ut.is_system():
            continue
        # 
        if ut.is_error_included(error):
            # print(ut.errors)
            y.append(1)
        else:
            y.append(0)

In [29]:
idf_path = "../../corpus/wiki/idf_wiki_v2.json"
with open(idf_path, "r") as f:
    idf_dict = json.load(f)

In [30]:
def is_under_idf_border_morpheme(text, idf_border):
    for token in mecab_tokenize(text):
        key = token
        if key in idf_dict:
            if idf_dict[key] > idf_border:
                return False
        else:
            return False
    return True

In [31]:
y_pred = []
border = 0.8

border_10 = 14.62592894814336
border_1 = 9.88253708481772
border_005 = 5.340630805533431

for conv in tqdm(convs):
    ngram_sets = []
    prev_sents = []
    for ut in conv:
        if not ut.is_system():
            continue
        utt = ut.utt
        doc = nlp(utt)
        y_pred.append(0)
        for sent in doc.sents:
            # idf が小さい場合は，スルーしましょう
            if is_under_idf_border_morpheme(sent.orth_, border_005):
                # print(sent)
                continue
            ngram_set = get_ngram_set(sent, N=3)
            # これまでのセットで重複が大きいものがあるかチェック！
            if check_repeat_rate(target=ngram_set, history=ngram_sets, border=0.8):
                # print(ut, ut.errors)
                y_pred[-1] = 1
            
            if y_pred[-1] == 0:
                 for prev in prev_sents:
                    if Levenshtein.ratio(sent.orth_, prev) >= 0.8:
                        y_pred[-1] = 1

            ngram_sets.append(ngram_set)
            prev_sents.append(sent.orth_)

100%|██████████| 200/200 [00:27<00:00,  7.28it/s]


In [32]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn import metrics
print('confusion matrix = \n', confusion_matrix(y_true=y, y_pred=y_pred))
print('accuracy = ', accuracy_score(y_true=y, y_pred=y_pred))
print('precision = ', precision_score(y_true=y, y_pred=y_pred))
print('recall = ', recall_score(y_true=y, y_pred=y_pred))
print('f1 score = ', f1_score(y_true=y, y_pred=y_pred))

confusion matrix = 
 [[2080   64]
 [  10   46]]
accuracy =  0.9663636363636363
precision =  0.41818181818181815
recall =  0.8214285714285714
f1 score =  0.5542168674698795


- ngram  border 0.8

        confusion matrix = 
        [[2131   21]
        [  29   19]]
        accuracy =  0.9772727272727273
        precision =  0.475
        recall =  0.3958333333333333
        f1 score =  0.4318181818181817

    - 再現率が欲しい

- Levenshtein bordr 0.8
    
        confusion matrix = 
        [[2102   50]
        [  19   29]]
        accuracy =  0.9686363636363636
        precision =  0.3670886075949367
        recall =  0.6041666666666666
        f1 score =  0.45669291338582674

    - 悪くないが，適合率が低い

- ngram  border 0.8 and Levenshtein bordr 0.8 and idf 005%
    - 仮定 : 「そうですよね」のように，全く情報量がない発話が繰り返された場合は不要な繰り返しの可能性が低い
        - idf 辞書の活用
    - 予想 : 誤検出が減る
    
            confusion matrix = 
            [[2091   61]
            [  15   33]]
            accuracy =  0.9654545454545455
            precision =  0.35106382978723405
            recall =  0.6875
            f1 score =  0.46478873239436624


    - 評価データでの実行
            
            confusion matrix = 
            [[2080   64]
            [  10   46]]
            accuracy =  0.9663636363636363
            precision =  0.41818181818181815
            recall =  0.8214285714285714
            f1 score =  0.5542168674698795


    - 誤検出が増えたが，未検出が減った
        - len(sent) <3 の中に，情報量があり，似た発話があるが不要ではない
        - len(sent) <3 の中に検出するべきエラーが存在した
    
    - 今後
        - どのような発話が雑談において不要ではないのかの検討が出来ていない
            - 共起辞書が活用できるか？
                - データを見た限り，そう簡単にはいかない
        - 非常に近い距離で，動詞や形容詞だけが違うという場合が検出出来ていない
            - 検出された発話中で，名詞や動詞などを比較するべきか

In [33]:
import Levenshtein

In [34]:
y_pred_2 = []
border = 0.8
for conv in tqdm(convs):
    # ngram_sets = []
    prev_sents = []
    for ut in conv:
        if not ut.is_system():
            continue
        utt = ut.utt
        doc = nlp(utt)
        y_pred_2.append(0)
        for sent in doc.sents:
            if len(sent) <= 3:
                continue
            for prev in prev_sents:
                if Levenshtein.ratio(sent.orth_, prev) >= border:
                    y_pred_2[-1] = 1
            # ngram_sets.append(ngram_set)
            prev_sents.append(sent.orth_)

100%|██████████| 200/200 [00:25<00:00,  7.72it/s]


In [35]:
score(y, y_pred_2)

confusion matrix = 
 [[2098   46]
 [  11   45]]
accuracy =  0.9740909090909091
precision =  0.4945054945054945
recall =  0.8035714285714286
f1 score =  0.6122448979591837


In [53]:
i = 0
for conv in convs:
    conv_list = []
    for ut in conv:
        conv_list.append(ut.utt)
        if not ut.is_system():
            continue
        # 本来エラーではないけどエラー扱い
        if y[i]==0 and y_pred[i]==1:
            print("本来エラーではないけどエラー扱い", ut.did)
            print(conv_list)
            print()
        # if y[i]==1 and y_pred[i]==0:
        #     print("本来エラーなのに非エラー扱い", ut.did)
        #     print(conv_list)
        #     print()
        if y[i]==1 and y_pred[i]==1:
            print("よく検出した！えらいぞ", ut.did)
            print(conv_list)
            print()
        
        i+=1

よく検出した！えらいぞ 1502868266
['こんにちは。ちょうど退屈してたんだ。', 'こんにちは。元気ですか？', '元気ですかは元気です', '元気そうでよかった。', '元気ですかは元気ですね']

よく検出した！えらいぞ 1502868266
['こんにちは。ちょうど退屈してたんだ。', 'こんにちは。元気ですか？', '元気ですかは元気です', '元気そうでよかった。', '元気ですかは元気ですね', 'いまなにしてたの？', 'もちろんですってのが元気ですかにいいですね', 'そうなんだ。どうして？', 'もちろんですってのが元気ですかにいいらしいですよ']

本来エラーではないけどエラー扱い 1502869359
['こんにちは！熱中症に気をつけて。', 'こんにちは。暑いから気をつけないといけませんね', '熱中症に気をつけか？？', '水をたくさん飲んでいますよ', '熱中症はいいですね', '良くないですよ', 'いつ', '暑い日はいつでも気をつけてください', '話は変わりますけど、アメリカの学校の夏休みは三ヶ月もあるって知ってました？', '知りませんでした！長いですね', '夏休みは欲しいですね', 'どれ位欲しいですか？', '夏休みは楽しいですね']

よく検出した！えらいぞ 1470623771
['こんにちは。海へ行きたいね。', '海はいいですね。泳ぎは得意ですか。', '泳ぎはいいですね', '気持ちいいですよね。', '泳ぎが上手ですね', 'ありがとう。クロールが得意なんです。', 'クロールは得意ですね', 'そう、スイミングで習ってるんです。', '朝からスイミングで泳ぎます？？', '泳ぎますよ。目が覚めていいですよ。', '目は覚めてるんですね。スイミングは楽しいですね', 'ですよね。そういえばオリンピックの水泳も楽しみですね。', 'スイミングがいいですね', '金メダルとれるといいですよね。', 'スイミングは楽しいですね']

よく検出した！えらいぞ 1502868964
['こんにちは。声かけてくれるのを待ってたんだ。', 'こんにちは。調子はどうですか？', '何か普段の生活で気をつけていることはありますか。', '健康には特に気を付けています。', '和食、洋食、中華どれが食べたいですか？', 

In [38]:
# 共起辞書の活用
ppmi_dataname = "../../corpus/collocation/ppmi_ntt1"
ppmi_matrix = np.load(ppmi_dataname+".npy")

In [40]:
dictname = "../../corpus/collocation/word_dict/ppmi_word_dict_ntt1.json"
with open(dictname, "r") as f:
    word_dict = json.load(f)

In [41]:
def ppmi(ppmi_matrix, word_dict, x, y):
    if x not in word_dict.keys():
        return 0
    elif y not in word_dict.keys():
        return 0

    x_id = word_dict[x]
    y_id = word_dict[y]
    return ppmi_matrix[x_id, y_id]

In [52]:
ppmi(ppmi_matrix, word_dict, "祭り", "掛け声")

0

In [51]:
clean_text("掛け声")

'掛け声'